In [1]:
# prerequisites
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

bs = 100
# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transforms.ToTensor(), download=False)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

In [2]:
class CVAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim, c_dim):
        super(CVAE, self).__init__()
        
        # encoder part
        self.fc1 = nn.Linear(x_dim + c_dim, h_dim1)
        self.fc2 = nn.Linear(h_dim1, h_dim2)
        self.fc31 = nn.Linear(h_dim2, z_dim)
        self.fc32 = nn.Linear(h_dim2, z_dim)
        # decoder part
        self.fc4 = nn.Linear(z_dim + c_dim, h_dim2)
        self.fc5 = nn.Linear(h_dim2, h_dim1)
        self.fc6 = nn.Linear(h_dim1, x_dim)
    
    def encoder(self, x, c):
        concat_input = torch.cat([x, c], 1)
        h = F.relu(self.fc1(concat_input))
        h = F.relu(self.fc2(h))
        return self.fc31(h), self.fc32(h)
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add(mu) # return z sample
    
    def decoder(self, z, c):
        concat_input = torch.cat([z, c], 1)
        h = F.relu(self.fc4(concat_input))
        h = F.relu(self.fc5(h))
        return F.sigmoid(self.fc6(h))
    
    def forward(self, x, c):
        mu, log_var = self.encoder(x.view(-1, 784), c)
        z = self.sampling(mu, log_var)
        return self.decoder(z, c), mu, log_var

# build model
cond_dim = train_loader.dataset.train_labels.unique().size(0)
cvae = CVAE(x_dim=784, h_dim1=512, h_dim2=256, z_dim=2, c_dim=cond_dim)
if torch.cuda.is_available():
    cvae.cuda()
else:
    cvae.cpu()

/Users/byunghunhwang/dev/flame-ssd-model/venv/lib/python3.11/site-packages/torchvision/datasets/mnist.py:65: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


In [3]:
optimizer = optim.Adam(cvae.parameters())
# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

# one-hot encoding
def one_hot(labels, class_size): 
    targets = torch.zeros(labels.size(0), class_size)
    for i, label in enumerate(labels):
        targets[i, label] = 1
    return Variable(targets)

In [4]:
def train(epoch):
    cvae.train()
    train_loss = 0
    for batch_idx, (data, cond) in enumerate(train_loader):
        data, cond = data.cpu(), one_hot(cond, cond_dim).cpu()
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = cvae(data, cond)
        loss = loss_function(recon_batch, data, mu, log_var)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [5]:
def test():
    cvae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, cond in test_loader:
            data, cond = data.cpu(), one_hot(cond, cond_dim).cpu()
            recon, mu, log_var = cvae(data, cond)
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [6]:
# train
for epoch in range(1, 51):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 544.341758
Train Epoch: 1 [10000/60000 (17%)]	Loss: 179.631191
Train Epoch: 1 [20000/60000 (33%)]	Loss: 155.215039
Train Epoch: 1 [30000/60000 (50%)]	Loss: 156.621416
Train Epoch: 1 [40000/60000 (67%)]	Loss: 149.539707
Train Epoch: 1 [50000/60000 (83%)]	Loss: 143.808545
====> Epoch: 1 Average loss: 163.0279
====> Test set loss: 141.2406
Train Epoch: 2 [0/60000 (0%)]	Loss: 154.150830
Train Epoch: 2 [10000/60000 (17%)]	Loss: 140.032090
Train Epoch: 2 [20000/60000 (33%)]	Loss: 141.778740
Train Epoch: 2 [30000/60000 (50%)]	Loss: 140.141094
Train Epoch: 2 [40000/60000 (67%)]	Loss: 136.568398
Train Epoch: 2 [50000/60000 (83%)]	Loss: 137.433770
====> Epoch: 2 Average loss: 138.6727
====> Test set loss: 136.4407
Train Epoch: 3 [0/60000 (0%)]	Loss: 133.293779
Train Epoch: 3 [10000/60000 (17%)]	Loss: 133.753838
Train Epoch: 3 [20000/60000 (33%)]	Loss: 134.501328
Train Epoch: 3 [30000/60000 (50%)]	Loss: 133.338652
Train Epoch: 3 [40000/60000 (67%)]	Loss: 134.26

In [ ]:
with torch.no_grad():
    # z = torch.randn(10, 2).cuda()
    # c = torch.eye(10).cuda()
    z = torch.randn(10, 2).cpu()
    c = torch.eye(10).cpu()
    

    sample = cvae.decoder(z, c)
    save_image(sample.view(10, 1, 28, 28), './samples/sample_' + '.png')